# Explorando a Requests

In [72]:
import requests
from dotenv import load_dotenv 
import os
import pandas as pd
import base64

In [40]:
# Lê o arquivo .env e carrega as variáveis no ambiente do Python
load_dotenv()

True

In [41]:
# Retorna uma lista dos eventos públicos mais recentes do GitHub
r = requests.get('https://api.github.com/events')

In [42]:
r

<Response [200]>

In [43]:
r.url

'https://api.github.com/events'

In [ ]:
# Retorna todo conteúdo em uma única string
r.text

In [ ]:
# Retorna todo conteúdo em formato json
r.json()

In [46]:
# Utilizando outro endpoint
r = requests.get('https://api.github.com/users/thaisbsts')
r.status_code

200

In [ ]:
# Mostra as informações sobre o usuário
r.json()

In [48]:
response_json = r.json()

In [49]:
# Extraindo a url que armazena as informações sobre repositórios
repos_url = response_json["repos_url"]

# Extraindo as informações da url de repositórios
response_repos = requests.get(repos_url)

In [50]:
# Tranformando em json e coletando o nº de elementos
repos = response_repos.json()
n_repos = len(repos)

In [51]:
# Extraindo o nome, nome de usuário e data de criação da conta
name = response_json["name"]
username = response_json["login"]
date_created = response_json["created_at"][0:10]

In [52]:
# Imprimindo as informações
print(f"O usuário {name} ({username}) criou sua conta no dia {date_created} e possui {n_repos} repositórios.")

O usuário Thais Bastos (thaisbsts) criou sua conta no dia 2024-06-07 e possui 14 repositórios.


# Obtendo dados dos repositórios

In [53]:
# Especificando a versão da API
headers = {'X-GitHub-Api-Version': '2022-11-28'}

In [54]:
api_base_url = 'https://api.github.com'
owner = 'amzn' # Username de quem vamos extrair os dados (Amazon)
url = f'{api_base_url}/users/{owner}/repos'

In [55]:
# Obtendo lista de repositórios da Amazon
response = requests.get(url, headers=headers)
response.status_code

200

In [56]:
''' 
Apesar do código abaixo retornar o número 30, a Amazon não possui apenas 30 repositórios.
Para acessar todos, é preciso considerar tanto a autenticação quanto a paginação da API.
'''

len(response.json())

30

## Autenticação
Caso eu não faça a autenticação, o limite de requisições por hora é 60.

In [57]:
# Token gerado no Github
access_token = os.getenv('API_KEY')
headers = { 'Authorization': 'Bearer ' + access_token,
           'X-GitHub-Api-Version': '2022-11-28'
           }

In [58]:
# Teste para verificar se está correto
response = requests.get(url, headers=headers)
response.status_code

200

## Paginação

In [59]:
api_base_url = 'https://api.github.com'
owner = 'amzn' # Username de quem vamos extrair os dados (Amazon)
url = f'{api_base_url}/users/{owner}/repos'

repos_list = [] # Lista de listas, cada lista é uma página
for page_num in range(1, 7):
    try:
        url_page = f'{url}?page={page_num}'
        response = requests.get(url_page, headers=headers)
        repos_list.append(response.json())
    except:
        repos_list.append(None)

In [60]:
soma = 0
for i in range(0, 6):
    soma += len(repos_list[i])

print(soma)

161


In [61]:
''' 
Atividade: encontrar a quantidade de seguidores da Amazon.
'''

username = 'amzn'
url = f"https://api.github.com/users/{username}/followers"
followers_list = []
page = 1

''' 
Como o número total de páginas é desconhecido, usarei um while True 
com a condição de parada sendo quando a página retornar zero itens (seguidores).
'''
while True: 
    url_page = f'{url}?page={page}'
    response = requests.get(url_page, headers=headers)
    followers = response.json()

    # Caso a lista esteja vazia, posso sair do laço pois todos os dados foram extraidos
    if len(followers)==0:
        break

    # Adicionando os seguidores a lista
    followers_list.append(followers)

    # Incrementa o valor de 'page' para a próxima requisição
    page += 1


In [62]:
# Somando e imprimindo o resultado
total = sum(len(page) for page in followers_list)
print(f"A Amazon tem {total} seguidores.")

A Amazon tem 2118 seguidores.


# Linguagens dos Repositórios

In [ ]:
# Extraindo apenas a linguagem usada em cada repositório
repos_languages = []

for page in repos_list:
    for repo in page:
        repos_languages.append(repo['language'])

# Nomes dos Repositórios

In [ ]:
# Extraindo os nomes dos repositórios 
repos_names = []
for page in repos_list:
    for repo in page:
        repos_names.append(repo["name"])

# Criando um DataFrame
Um dataframe é uma representação programática de uma tabela que facilita a visualização, filtragem e manipulação de dados. Por isso, vou converter o conteúdo extraído da API para essa estrutura.

In [69]:
''' 
Criando um DataFrame e adicionando duas colunas com os dados
que foram extraídos anteriormente (nomes dos repositórios e a
linguagem usada em cada um).
'''

dados_amzn = pd.DataFrame()
dados_amzn["repository_name"] = repos_names
dados_amzn["language"] = repos_languages

In [70]:
display(dados_amzn)

,repository_name,language
0,.github,None
1,ads-advanced-tools-docs,Jupyter Notebook
2,ads-pao-amznjs-gtm-server-side-template,Smarty
3,ads-pao-amznjs-gtm-template,Smarty
4,alexa-coho,JavaScript
...,...,...
156,zeek-plugin-enip,Zeek
157,zeek-plugin-profinet,Zeek
158,zeek-plugin-s7comm,Zeek
159,zeek-plugin-tds,Zeek


In [ ]:
# Salvando os dados em um csv
dados_amzn.to_csv('amazon.csv')

# Disponibilizando o conteúdo
Por fim, é necessário disponibilizar os dados extraídos para que outros times da empresa onde trabalhamos também tenham acesso. Para isso, pode-se criar um repositório no Github com a ajuda da própria biblioteca Requests.

In [ ]:
'''
Não é necessário especificar meu username pois a API reconhece quem está fazendo 
a requisição com base no token de acesso fornecido no cabeçalho/headers.
'''

'''
api_base_url = 'https://api.github.com'
url = f'{api_base_url}/user/repos' 

# Dados do repositório
data = {
    'name': 'linguagens-utilizadas',
    'description': 'Repositorio com as linguagens de prog da Amazon',
    'private': False
}

response = requests.post(url, json=data, headers=headers)

# O repositório foi criado. Agora é necessário fazer o upload do arquivo.
# Segundo a documentação, o arquivo precisa estar em Base64, então é necessário transformar o csv.

with open('amazon.csv', 'rb') as file:
    file_content = file.read()

encoded_content = base64.b64encode(file_content)

api_base_url = 'https://api.github.com'
username = 'thaisbsts'
repo = 'linguagens-utilizadas'
path = 'amazon.csv'

url = f'{api_base_url}/repos/{username}/{repo}/contents/{path}'

data = {
    'message': 'add: file with repositories languages',
    'content': encoded_content.decode('utf-8')
}

response = requests.put(url, json=data, headers=headers)
'''